# Download Protocol List

In [16]:
import requests
import pandas as pd

url = "https://api.llama.fi/protocols"
response = requests.get(url)
data = response.json()

df_protocol = pd.DataFrame(data)
df_protocol["protocol"] = df_protocol["name"].apply(lambda x: x.lower().replace(" ", "-"))
df_protocol.to_excel("protocol.xlsx", index=False)
print(df_protocol.shape, df_protocol.columns)
df_protocol.head()


(4772, 49) Index(['id', 'name', 'address', 'symbol', 'url', 'description', 'chain',
       'logo', 'audits', 'audit_note', 'gecko_id', 'cmcId', 'category',
       'chains', 'module', 'twitter', 'forkedFrom', 'oracles', 'listedAt',
       'methodology', 'slug', 'tvl', 'chainTvls', 'change_1h', 'change_1d',
       'change_7d', 'tokenBreakdowns', 'mcap', 'referralUrl', 'treasury',
       'audit_links', 'openSource', 'governanceID', 'github', 'hallmarks',
       'parentProtocol', 'wrongLiquidity', 'staking', 'stablecoins',
       'oraclesByChain', 'pool2', 'assetToken', 'language',
       'tokensExcludedFromParent', 'misrepresentedTokens', 'deadUrl', 'rugged',
       'deadFrom', 'protocol'],
      dtype='object')


,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,oraclesByChain,pool2,assetToken,language,tokensExcludedFromParent,misrepresentedTokens,deadUrl,rugged,deadFrom,protocol
0,2269,Binance CEX,None,-,https://www.binance.com,Binance is a cryptocurrency exchange which is ...,Multi-Chain,https://icons.llama.fi/binance-cex.jpg,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,binance-cex
1,182,Lido,0x5a98fcbea516cf06857215779fd812ca3bef1b32,LDO,https://lido.fi/,Liquid staking for Ethereum and Polygon. Daily...,Multi-Chain,https://icons.llama.fi/lido.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lido
2,2272,OKX,None,-,https://www.okx.com,"OKX, formerly known as OKEx, is a Seychelles-b...",Multi-Chain,https://icons.llama.fi/okx.jpg,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,okx
3,2275,Bitfinex,None,-,https://www.bitfinex.com,Bitfinex facilitates a graphical trading exper...,Multi-Chain,https://icons.llama.fi/bitfinex.png,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bitfinex
4,3732,Robinhood,None,-,https://robinhood.com,Democratizing finance for all. Crypto trading:...,Multi-Chain,https://icons.llama.fi/robinhood.jpg,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,robinhood


In [43]:
# zeroTvl.txt 에 있는 protocol의 name과 category 가져오기

with open("zeroTvl.txt", "r") as f:
    lines = f.readlines()
    lines = [x.strip() for x in lines]

df = pd.read_excel("protocol.xlsx")

for line in lines:
    temp = df[df["protocol"] == line]
    print(temp["category"].values[0], line, temp["name"].values[0], sep=" | ")


DEX Aggregator | 0x | 0x
Services | abachi | Abachi
Chain | acala | Acala
DEX Aggregator | airswap | AirSwap
Yield | apecoin | ApeCoin
Chain | aptos | Aptos
Services | aragon | Aragon
Services | arbitrum-dao | Arbitrum DAO
Chain | arbitrum | Arbitrum
Services | arkham | Arkham
Yield | astral-financial | Astral Financial
Services | avalanche | Avalanche
DEX Aggregator | avnu | AVNU
Services | bad-idea-ai | Bad Idea AI
Telegram Bot | banana-gun | Banana Gun
Reserve Currency | bank-of-cronos-ohm | Bank of Cronos OHM
Services | bankless | Bankless
Derivatives | basedmarkets | basedmarkets
Services | basepaint | BasePaint
Services | basic-attention | Basic Attention
Dexes | bisq | Bisq
Chain | bitcoin | Bitcoin
DEX Aggregator | bitget-swap | Bitget Swap
Services | blast-api | Blast API
Telegram Bot | blazebot | BlazeBot
Dexes | bonedex | BoneDex
Services | braindao | BrainDAO
Chain | canto | Canto
Chain | celestia | Celestia
Oracle | chainlink-keepers | Chainlink Keepers
Oracle | chainlink-

# Download Protocol Data

In [17]:
import os
os.makedirs("html", exist_ok=True)
downloaded = [v[:-5] for v in os.listdir("html")]
downloaded[:5]

['0.exchange', '01', '0vix', '0x-aggregator', '0x-nodes']

In [ ]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

options = Options()
# React로 만들어진 웹 페이지의 경우 headless 모드에서 
# javascript unenable로 인하여 데이터를 가져오지 못함
# options.add_argument("--headless")  
driver = webdriver.Chrome(options=options)
url = "https://defillama.com/protocol/"

for v in df_protocol["protocol"]:
    if v in downloaded:
        continue
    driver.get(url+v)
    sleep(15)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    next_data_script = soup.find('script', id="__NEXT_DATA__")
    try:
        data = next_data_script.string
        data = json.loads(data)
        if "error" in data["page"]:
            print(v, data["err"])
            continue
        if next_data_script.string:
            try:
                with open(f"html/{v}.json", "w") as f:
                    f.write(data)
            except UnicodeEncodeError as e:
                with open(f"html/{v}.json", "w", encoding="utf8") as f:
                    f.write(data)
        else:
            print(v, "id='__NEXT_DATA__'를 가진 script 태그를 찾을 수 없습니다.")
    except Exception as e:
        print(v, e)

driver.quit() # 1847 min at 20s sleep

# Extract TVL

In [7]:
import os
import json
import pandas as pd
from datetime import datetime

os.makedirs("df", exist_ok=True)

failed = {
    "Empty": [],
    "OneTvl": [],
    "UnicodeDecodeError": [],
    "KeyError": [],
    "Other": [],
}

In [8]:
def generate_df(times, network):
    times = times["tvl"]
    df = pd.DataFrame(times)
    df["date"] = df["date"].apply(lambda x: datetime.fromtimestamp(x)) 
    df[network] = df["totalLiquidityUSD"]
    return df[["date", network]]

def is_filled(df_hist, df_not):
    global failed

    if df_hist is None and df_not is None:
        print(f"Empty tvl in {v}")
        with open("emptyTvl.txt", "a") as f:
            f.write(f"{v[:-5]}\n")
        return True
    elif df_hist is None or df_not is None:
        print(f"Only one tvl in {v}: {df_hist.shape if df_hist else df_not.shape}")
        failed["OneTvl"].append(v)
        return True
    return False

def is_identical(df_hist, df_not):
    res = [
        (c, (df_hist[c] != df_not[c]).sum())
        for c in df_hist.columns
    ]
    result = (sum([v[-1] for v in res]) > 0)
    if result:
        print(f"Different in {v}")
        print(res)
    return not result

In [9]:
# 다운로드된 파일 목록 가져오기
# downloaded = [v[:-5] for v in os.listdir("html")]

# emptyTvl.txt 파일을 초기화
with open("emptyTvl.txt", "w") as f:
    f.write("")


for v in os.listdir("html"):

    # 다운로드된 파일 목록에 있으면 다음 파일로 넘어감
    # if v[:-5] in downloaded:
    #     continue

    try: 
        data = None
        try:
            with open(f"html/{v}", "r", encoding='utf8') as f:
                data = f.read()
        except UnicodeDecodeError as e:
            try:
                with open(f"html/{v}", "r", encoding='cp949') as f:
                    data = f.read()
            except Exception as e:
                raise e
            
        if not data:
            print(f"Empty in {v}")
            failed["Empty"].append(v)
            continue
        data = json.loads(data)
        data = data["props"]["pageProps"]["protocolData"]

        tvl = {
            "history": {
                "data": data["historicalChainTvls"],
                "df": None,
            },
            "nothist": {
                "data": data["chainTvls"],
                "df": None,
            }
        }
        
        for k in tvl.keys():
            dfs = [
                generate_df(times, network) 
                for network, times in tvl[k]["data"].items()
                if times and times["tvl"]
            ]
            
            if len(dfs)==1:
                tvl[k]["df"] = dfs[0].fillna(0)
            elif dfs:
                result = dfs[0]
                for d in dfs[1:]:
                    result = pd.merge(result, d, on="date", how="outer")
                tvl[k]["df"] = result.fillna(0)

        if is_filled(tvl["history"]["df"], tvl["nothist"]["df"]):
            continue

        if is_identical(tvl["history"]["df"], tvl["nothist"]["df"]):
            tvl["nothist"]["df"].to_csv(
                f"df/{v.replace('.json', '.csv')}", index=False)

    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError in {v}: {e}")
        failed["UnicodeDecodeError"].append(v)
    except KeyError as e:
        print(f"KeyError in {v}: {e}")
        failed["KeyError"].append(v)
    except Exception as e:
        print(f"Error in {v}: {e}")
        failed["Other"].append(v)

Empty tvl in 0x-aggregator.json
Empty tvl in 7k-aggregator.json
Empty tvl in aftermath-aggregator.json
Empty tvl in aggre.json
Empty tvl in aimbot.json
Empty tvl in akka.json
Empty tvl in algebra-integral.json
Empty tvl in altlayer.json
Empty tvl in ape-jupiter.json
Empty tvl in arcane-dex.json
Empty tvl in basecamp.json
Empty tvl in beam.json
Empty tvl in bear-vs-bull.json
Empty tvl in bebop.json
Empty tvl in big-time.json
Empty tvl in bitoro-network.json
Empty tvl in bitscrunch.json
Empty tvl in bittensor.json
Empty tvl in bloxroute.json
Empty tvl in bonkbot.json
Empty tvl in bouncebit.json
Empty tvl in bunnyfi.json
Empty tvl in cardano.json
Empty tvl in cellula.json
Empty tvl in cetus-aggregator.json
Empty tvl in chainlink-ccip.json
Empty tvl in cheelee.json
Empty tvl in clusters.json
Empty tvl in coinbase-commerce.json
Empty tvl in commonwealth.json
Empty tvl in conveyor.json
Empty tvl in cyberconnect.json
Empty tvl in defiway.json
Empty tvl in dex-screener.json
Empty tvl in dexhun

In [10]:
failed

{'Empty': [],
 'OneTvl': [],
 'UnicodeDecodeError': [],
 'KeyError': [],
 'Other': []}

In [11]:
os.makedirs("result", exist_ok=True)
files = os.listdir("df")

with open("zeroTvl.txt", "w") as f:
    f.write("")

for d in files:
    df = pd.read_csv(f"df/{d}")
    df["TotalTvl"] = df.iloc[:, 1:].sum(axis=1)
    if df["TotalTvl"].sum() == 0:
        with open("zeroTvl.txt", "a") as f:
            f.write(f"{d[:-4]}\n")
        continue
    name = d[:-4]
    df[name] = df["TotalTvl"]
    df[["date", name]].to_csv(f"result/{d}", index=False)
    df.to_csv(f"df/{d}", index=False)